In [27]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

# 1️⃣ Load dataset
df = pd.read_csv(r"data/CAR_EVALUATION.csv")
# print(df.head())

TARGET = "Target"

X = df.drop(columns=[TARGET])
y = df[TARGET]


X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, train_size=1000, shuffle=True, random_state=42, stratify=y
)

X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=428, shuffle=True, random_state=42, stratify=y_temp
)

cat_cols = X_train.select_dtypes(include=["object"]).columns.tolist()
print("📋 Categorical columns:", cat_cols)

ohe = OneHotEncoder()

X_train_enc = ohe.fit_transform(X_train)
X_val_enc = ohe.transform(X_val)
X_test_enc = ohe.transform(X_test)


le = LabelEncoder()
y_train_enc = le.fit_transform(y_train)
y_val_enc   = le.transform(y_val)
y_test_enc  = le.transform(y_test)

df.head()

📋 Categorical columns: ['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety']


,buying,maint,doors,persons,lug_boot,safety,Target
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc
3,vhigh,vhigh,2,2,med,low,unacc
4,vhigh,vhigh,2,2,med,med,unacc
